In [1]:
import pandas as pd

In [2]:
filename = "./data.csv"

target_value = "value"
lst_feat = ["f1", "f2"]

divide_col = "stage"
bias_col = "bias"
spec = 5

In [3]:
def concat_cols(row: pd.Series):
    return "_".join(row.values.astype(str))

In [4]:
df_data = pd.read_csv(filename)
df_data["comb"] = df_data[lst_feat].apply(concat_cols, axis=1)
lst_stage = sorted(df_data[divide_col].unique())

In [6]:
for stage in lst_stage:
    cond_stage = df_data[divide_col] == stage
    df_stage = df_data[cond_stage]
    grouped_df = df_stage.groupby("comb", as_index=False)[target_value].agg(
        {
            bias_col: lambda x: abs(max(x) - min(x)),
            "num": "count",
        }
    )
    total_num = grouped_df["num"].sum()
    grouped_df["ratio"] = grouped_df["num"] / total_num    
    cond_spec = grouped_df[bias_col] > spec
    df_oos = grouped_df[cond_spec]
    
    print(f"stage: {stage}")
    print(grouped_df)
    
    if df_oos.shape[0] > 0:
        print("oos:")
        print(df_oos)

stage: P1
   comb  bias  num     ratio
0  A_aa     0    1  0.333333
1  A_bb     1    2  0.666667
stage: P2
   comb  bias  num     ratio
0  A_bb    12    3  0.333333
1  B_aa     1    2  0.222222
2  B_bb     0    1  0.111111
3  C_bb     9    3  0.333333
oos:
   comb  bias  num     ratio
0  A_bb    12    3  0.333333
3  C_bb     9    3  0.333333
